In [1]:
import re
from scipy import stats
import os
import pandas as pd # package for high-performance, easy-to-use data structures and data analysis
import numpy as np  # fundamental package for acientific computing with python
import matplotlib 
from matplotlib import pyplot as plt # for plotting
import seaborn as sns # for making plots with seaborn
color = sns.color_palette()
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot

from plotly import tools

init_notebook_mode(connected=True)

import config as cfg
import utils

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline 

### Data exploration
Visualization code reference:https://www.kaggle.com/shivamb/homecreditrisk-extensive-eda-baseline-0-772

In [2]:
# The function to plot the distribution of the categorical values Horizontaly 
def bar_hor(df,  col, title, color, w=None, h=None, lm=0,  limit=100, return_trace=False, rev=False, xlb=False):
    cnt_srs = df[col].value_counts()
    yy = cnt_srs.head(limit).index[::-1]
    xx = cnt_srs.head(limit).values[::-1]
    if rev:
        yy = cnt_srs.tail(limit).index[::-1]
        xx = cnt_srs.tail(limit).values[::-1]
    if xlb:#????
        trace = go.Bar(y=xlb, x=xx,orientation='h', marker=dict(color=color))
    else:
        trace = go.Bar(y=yy, x=xx,orientation='h', marker=dict(color=color))
    if return_trace:
        return trace
    layout = dict(title=title, margin=dict(l=lm), width=w, height=h)
    data = [trace]
    fig = go.Figure(data=data, layout=layout)
    iplot(fig)

    
# The function to get the distribution of the categories according to the target
#(target de dtype=bool? or np.int8?)
def gp_1(df, col, title):
    df0 = df[df['label']==0]
    df1 = df[df['label']==1]
    a1 = df0[col].value_counts()
    b1 = df1[col].value_counts()
    
    total = dict(df[col].value_counts())
    x0 = a1.index
    x1 = b1.index
    
    y0 = [float(x) for i,x in enumerate(a1.values)]
    y1 = [float(x) for i,x in enumerate(b1.values)]
    
    y0, x0 = zip(*sorted(zip(y0, x0), reverse=True))
    y1, x1 = zip(*sorted(zip(y1, x1), reverse=True))
    print(x0[:5],y0[:5])
    print(x1[:5],y1[:5])
    trace1 = go.Bar(x=x0, y=y0, name="Target : 0", marker=dict(color="#96D38C"),hoverinfo="all")
    trace2 = go.Bar(x=x1, y=y1, name="Target : 1", marker=dict(color="#FEBFB3"),hoverinfo="all")
    
    return trace1, trace2
def catAndTrgt(df, col, limit=100):
    tr0 = bar_hor(df, col, "Distribution of " + col, "#f975ae", w=700, lm=100, limit=limit, return_trace=True)
    tr1, tr2 = gp_1(df, col, "Distribution of Target with " + col)
    
    fig = tools.make_subplots(rows=1, cols=3, print_grid=False, 
                             subplot_titles=[col+" Distribution", "count of target=0", "count of target=1"])
    fig.append_trace(tr0, 1, 1);
    fig.append_trace(tr1, 1, 2);
    fig.append_trace(tr2, 1, 3);
    fig['layout'].update(height=350, showlegend=False, margin=dict(l=50));
    iplot(fig);


## Read in data

In [3]:
print(os.listdir('./data'))

['口碑 路虎 总.xlsx', 'luhu_comments.xlsx', '路虎口碑所有用户.xlsx', '.DS_Store', '路虎口碑用户.xlsx', '路虎车商城.xlsx', 'luhu_koubei_user.xlsx', '路虎口碑用户-1.xlsx', '捷豹车商城.xlsx', '路虎口碑评论人.xlsx', '~$捷豹口碑用户.xlsx', '捷豹口碑用户.xlsx', '路虎帖子评论人.xlsx', 'luhu_comment_user.xlsx', '口碑 捷豹 评论.xlsx', 'jiebao_user_koubei.xlsx', 'luhu_all_user_0.xlsx', '.ipynb_checkpoints', '路虎口碑评论人-1.xlsx', 'jiebao_all_user_0.xlsx', 'jiebao_all_user_processed.xlsx', '捷豹口碑评论人.xlsx', '口碑 路虎 评论.xlsx', '~$路虎口碑所有用户.xlsx', 'jlr_all_user_processed.xlsx', '口碑 捷豹 总 252.xlsx', 'luhu_all_user_1.xlsx']


In [4]:
jiepao_path = './data/jiebao_all_user_0.xlsx'
luhu_path = './data/luhu_all_user_1.xlsx'
jiepao_df = pd.read_excel(jiepao_path)
luhu_df = pd.read_excel(luhu_path)

In [5]:
print(jiepao_df.keys() == luhu_df.keys())

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True]


In [6]:
jlr_df = jiepao_df.append(luhu_df)
print(jiepao_df.shape, luhu_df.shape, jlr_df.shape)

jlr_df.to_excel('jiebao_luhu_all_use.xlsx', encoding='utf-8')

(2981, 19) (2574, 19) (5555, 19)


### Frature engineering

In [7]:
print(jlr_df.keys())
jlr_df.head(1)

Index(['user_id', 'follow_count', 'fans_count', 'gender', 'birthday',
       'location', 'level', 'registration_time', 'properties', 'mileage',
       'post', 'cars', 'koubei_post', 'gas_mileage', 'car_friend_zone',
       'label', 'car_like', 'excellent_post_count', 'all_post_count'],
      dtype='object')


,user_id,follow_count,fans_count,gender,birthday,location,level,registration_time,properties,mileage,post,cars,koubei_post,gas_mileage,car_friend_zone,label,car_like,excellent_post_count,all_post_count
0,oden123,0,0,man,NaT,江西 宜春,1,2018-03-26,普通用户,4430,NaN,['捷豹 捷豹XFL 2018款 XFL 2.0T 200PS 精英版'],"[{'title': '敲黑板了！2018款xfl精英版2个月的使用感受，要考的', 'ur...","[{'车型': '捷豹 捷豹XFL 2018款 XFL 2.0T 200PS 精英版', '...",NaN,1,宝马5系,0,0


In [8]:
X = jlr_df.copy()
# X = jlr_df.drop(columns=['user_id',
#                          'birthday',
#                          'post', 
#                          'cars',
#                          'koubei_post',
#                          'car_friend_zone',
#                               ])
# Y = list(jlr_df['label'])

In [9]:
# X.location.unique()
# X.car_like.unique()
X.properties.unique()

array(['普通用户', '关禁闭', 'SLS赛威论坛版主', '捷豹F-PACE论坛版主', '编辑', '捷豹XF/XFL论坛版主',
       'YARiS L 致炫论坛版主', '宝马i3论坛版主', '大7 SUV论坛版主', '神行者2论坛版主'],
      dtype=object)

In [10]:
X['follow_count'] = X['follow_count'].astype('uint16')
X['fans_count'] = X['fans_count'].astype('uint16')

In [11]:
utils.catAndTrgt(df=X, col='follow_count', limit=20)

In [12]:
catAndTrgt(df=X, col='follow_count', limit=10)

(0, 1, 2, 3, 4) (1552.0, 589.0, 374.0, 263.0, 211.0)
(0, 1, 2, 3, 5) (206.0, 61.0, 58.0, 36.0, 30.0)


In [13]:
utils.catAndTrgt(df=X, col='fans_count', limit=20)

In [14]:
X['gender'] = X.apply(lambda row: utils.gender2bool(row), axis=1)
X['gender'] = X['gender'].astype('uint8')

In [15]:
utils.catAndTrgt(jlr_df, 'gender')

In [16]:
catAndTrgt(jlr_df, 'gender')

('man', 'woman') (4706.0, 124.0)
('man', 'woman') (691.0, 34.0)


In [17]:
X['location'] = X.apply (lambda row: utils.loc2value(row), axis=1)
X['location'] = X['location'].astype('uint16')

In [18]:
utils.catAndTrgt(jlr_df, 'location',10)

In [19]:
catAndTrgt(jlr_df, 'location',10)

('北京 ', '广东 广州', '上海 ', '四川 成都', '浙江 杭州') (542.0, 214.0, 200.0, 187.0, 159.0)
('北京 ', '上海 ', '浙江 杭州', '四川 成都', '广东 广州') (73.0, 40.0, 33.0, 31.0, 29.0)


In [20]:
X['level'] = X['level'].astype('uint8')
X.level.unique()

array([1, 2, 3, 4, 5, 6], dtype=uint64)

In [21]:
utils.catAndTrgt(jlr_df, 'level')

In [22]:
catAndTrgt(jlr_df, 'level')

(1, 2, 3, 4, 5) (4372.0, 330.0, 100.0, 21.0, 7.0)
(1, 2, 3, 4, 6) (597.0, 92.0, 27.0, 8.0, 1.0)


In [23]:
X['regis_year'] = X['registration_time'].dt.year.astype('uint16')
X['regis_month'] = X['registration_time'].dt.month.astype('uint8')
X['regis_day'] = X['registration_time'].dt.month.astype('uint8')

In [24]:
X['date'] = pd.to_datetime(X['registration_time'])    
X['date_delta'] = (X['date'] - X['date'].min())  / np.timedelta64(1,'D')

In [25]:
utils.catAndTrgt(X, 'date_delta', limit=10)

In [26]:
catAndTrgt(X, 'date_delta', limit=10)

(4320.0, 4462.0, 4329.0, 4326.0, 4266.0) (15.0, 14.0, 12.0, 12.0, 12.0)
(4397.0, 3289.0, 4494.0, 4411.0, 4406.0) (4.0, 4.0, 3.0, 3.0, 3.0)


In [27]:
utils.catAndTrgt(X, 'registration_time', limit=10)

In [28]:
catAndTrgt(X, 'registration_time', limit=10)

(Timestamp('2017-09-05 00:00:00'), Timestamp('2018-01-25 00:00:00'), Timestamp('2017-09-14 00:00:00'), Timestamp('2017-09-11 00:00:00'), Timestamp('2017-07-13 00:00:00')) (15.0, 14.0, 12.0, 12.0, 12.0)
(Timestamp('2017-11-21 00:00:00'), Timestamp('2014-11-09 00:00:00'), Timestamp('2018-02-26 00:00:00'), Timestamp('2017-12-05 00:00:00'), Timestamp('2017-11-30 00:00:00')) (4.0, 4.0, 3.0, 3.0, 3.0)


In [29]:
# Group year and month as feature
# X['year_month'] = pd.to_datetime(X[['regis_day','regis_month','regis_year']])

In [30]:
# X.regis_time_relative.iloc[0]

In [31]:
# np.array(X['regis_time_relative'])/(10.**11)

In [32]:
X['properties'] = X.apply(lambda row: utils.property2value(row), axis=1)
X['properties'] = X['properties'].astype('uint8')
#new_df.head()

In [33]:
utils.catAndTrgt(jlr_df, 'properties')

In [34]:
catAndTrgt(jlr_df, 'properties')

('普通用户', '关禁闭', '编辑', '捷豹XF/XFL论坛版主', '捷豹F-PACE论坛版主') (4740.0, 75.0, 8.0, 2.0, 2.0)
('普通用户', '关禁闭', '编辑', '捷豹F-PACE论坛版主', '宝马i3论坛版主') (697.0, 23.0, 1.0, 1.0, 1.0)


In [35]:
X['car_like'] = X.apply(lambda row: utils.cars_like2value(row), axis=1)
X['car_like'] = X['car_like'].astype('uint16')

In [36]:
utils.catAndTrgt(jlr_df, 'car_like')

In [37]:
catAndTrgt(jlr_df, 'car_like')

('宝马3系', '宝马5系', '博越', '捷豹F-PACE', '捷豹XFL') (207.0, 162.0, 133.0, 107.0, 94.0)
('神行者2', '发现神行(进口)', '发现', '捷豹F-PACE', '揽胜运动版') (113.0, 47.0, 43.0, 41.0, 27.0)


In [38]:
X['excellent_post_count'] = X['excellent_post_count'].astype('uint16')
X['all_post_count'] = X['all_post_count'].astype('uint16')

In [39]:
utils.catAndTrgt(jlr_df, 'excellent_post_count',limit=10)

In [40]:
catAndTrgt(jlr_df, 'excellent_post_count',limit=10)

(0, 1, 2, 3, 4) (4477.0, 182.0, 65.0, 48.0, 17.0)
(0, 1, 2, 3, 7) (570.0, 89.0, 23.0, 17.0, 5.0)


In [41]:
utils.catAndTrgt(jlr_df, 'all_post_count',limit=10)

In [42]:
catAndTrgt(jlr_df, 'all_post_count',limit=10)

(0, 1, 2, 3, 4) (1969.0, 543.0, 313.0, 215.0, 212.0)
(0, 1, 2, 3, 4) (179.0, 88.0, 46.0, 44.0, 33.0)


In [43]:
X['label'] = X['label'].astype('bool')

In [44]:
utils.exploreCat(jlr_df, 'label')

In [45]:
X.to_excel('./data/jlr_all_user_processed.xlsx', encoding='utf-8')

In [46]:
X.head()

,user_id,follow_count,fans_count,gender,birthday,location,level,registration_time,properties,mileage,...,car_friend_zone,label,car_like,excellent_post_count,all_post_count,regis_year,regis_month,regis_day,date,date_delta
0,oden123,0,0,2,NaT,1,1,2018-03-26,1,4430,...,NaN,True,1,0,0,2018,3,3,2018-03-26,4522.0
1,生活1934626,2,4,2,NaT,2,1,2012-05-07,1,4270,...,"[{'名称': '汽车之家甘肃论坛车友会', '人数': 672, '创建时间': '201...",True,2,0,0,2012,5,5,2012-05-07,2373.0
2,wangzi1125,0,1,2,NaT,3,1,2017-06-18,1,2090,...,NaN,True,3,0,1,2017,6,6,2017-06-18,4241.0
3,房产专家谢广财,3,2,2,NaT,4,1,2014-02-27,1,1120,...,NaN,True,4,0,0,2014,2,2,2014-02-27,3034.0
4,南宫晗笑,4,5,2,NaT,5,1,2017-07-03,1,740,...,NaN,True,1,0,14,2017,7,7,2017-07-03,4256.0
